In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [ ]:
!apt install libopenmpi-dev
!pip install mpi4py

In [ ]:
%%cu
#include <vector>
#include <thread>
#include <algorithm>
#include <functional>
#include <iostream>

void do_work(unsigned id)
{}

void f()
{
    std::vector<std::thread> threads;
    for(unsigned i=0;i<20;++i)
    {
        threads.push_back(std::thread(do_work,i));
    }
    std::for_each(threads.begin(),threads.end(),
        std::mem_fn(&std::thread::join));
}

int main()
{
    f();
    std::cout << "Hello world\n";
    return 0;
}


Hello world



In [ ]:
%%cu
#include <thread>
#include <iostream>



void do_something(int& i) {
    ++i;
    //std::cout<<"i = "<<i<<std::endl;
}

struct func {
    int &i;
    
    func(int& i_) : i(i_) { }

    void operator()() {
        for(unsigned j=0; j<1000000; ++j){
            do_something(i);
        }
    }
};

void do_something_in_current_thread() { }

void F() {
    int some_local_state = 0;
    func my_func(some_local_state);
    std::thread t(my_func);

    try{
        do_something_in_current_thread();
    }
    catch(...){
        t.join();
        throw;
    }
    t.join();
}

int main()
{
    F();
}

In [ ]:
%%cu
#include <mutex>
#include <condition_variable>
#include <queue>
#include <memory>

namespace messaging
{
    struct message_base
    {
        virtual ~message_base()
        {}
    };

    template<typename Msg>
    struct wrapped_message:
    message_base
    {
        Msg contents;
        explicit wrapped_message(Msg const& contents_):
            contents(contents_)
        {}
    };

    class queue
    {
        std::mutex m;
        std::condition_variable c;
        std::queue<std::shared_ptr<message_base> > q;

    public:

        template<typename T>
        void push(T const& msg)
        {
            std::lock_guard<std::mutex> lk(m);
            q.push(std::make_shared<wrapped_message<T> >(msg));
            c.notify_all();
        }

        std::shared_ptr<message_base> wait_and_pop()
        {
            std::unique_lock<std::mutex> lk(m);
            c.wait(lk,[&]{return !q.empty();});
            auto res=q.front();
            q.pop();
            return res;
        }
    };
}

In [ ]:
%%cu
namespace messaging
{
    class sender
    {
        queue*q;

    public:

        sender():
        q(nullptr) {}

        explicit sender(queue*q_) : q(q_)
        {}

        template<typename Message>
        void send(Message const& msg)
        {
            if(q)
            {
                q->push(msg);
            }
        }
    };
}

In [ ]:
%%cu

namespace messaging
{
    class receiver
    {
        queue q;

    public:
     
        operator sender()
        {
            return sender(&q);
        }

        dispatcher wait()
        {
            return dispatcher(&q);
        }
    };
}

In [ ]:
%%cu
namespace messaging
{
    class close_queue
    {};

    class dispatcher
    {
        queue* q;
        bool chained;

        dispatcher(dispatcher const&)=delete;
        dispatcher& operator=(dispatcher const&)=delete;

        template<typename Dispatcher, typename Msg, typename Func>
        friend class TemplateDispatcher;

        void wait_and_dispatch()
        {
            for(;;)
            {
                auto msg=q->wait_and_pop();
                dispatch(msg);
            }
        }


        bool dispatch(std::shared_ptr<message_base> const& msg)
        {

            if(dynamic_cast<wrapped_message<close_queue>*>(msg.get()))
            {
                throw close_queue();
            }
            return false;
        }

    public:

        dispatcher(dispatcher&& other) : q(other.q),chained(other.chained)
        {
            other.chained=true;
        }

        explicit dispatcher(queue* q_) : q(q_),chained(false)
        {}

        template<typename Message,typename Func>
        TemplateDispatcher<dispatcher,Message,Func>

        handle(Func&& f)
        {
            return TemplateDispatcher<dispatcher,Message,Func>(
                q,this,std::forward<Func>(f));
        }

        ~dispatcher() noexcept(false)
        {
            if(!chained)
            {
                wait_and_dispatch();
            }
        }
    };
}